### Selected Word Group In Youtube Sentence

In [1]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish"

# parameter
file = "Basics Cover the Most Master File Threegram Selected Sample Word"  # without xlsx extention
sheets = "Sheet1"  # 2 gram target, 2 gram hybrid
file_ext = "Threegram_Remain_Word2"
first_column = 0  # start column location number
end_column = 1
time_shift = 0.6  # 0.6
sample_num = 10  # 6

# youtube sentence condition
sent_time_start = False  # True, False
sent_time_start_time = 700  # sn
random_sample = True  # True, False

In [3]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/7-Selected Word Group In Youtube Sentence/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [4]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [6]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    
    return  df_word_count

In [7]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [8]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [9]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3856314,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3856315,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3856316,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3856317,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [10]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [11]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3856314,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3856315,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3856316,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3856317,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [12]:
if sent_time_start:
    df_youtube_select = df_youtube_sentence[df_youtube_sentence["start_time"] <= sent_time_start_time]
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    df_youtube_select = df_youtube_sentence

if random_sample:
    df_youtube_select = df_youtube_select.sample(len(df_youtube_select))
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    pass

df_youtube_select

,start_time,end_time,sentence,video_id
0,1711.520,1713.740,kardeşim benim buraya gelişimdeki amaç,OImI_S8C2vo
1,6775.440,6779.540,ağam bu terliklerle gidilir mi çizmelerin nerede,LeIuG8Wq2WY
2,8398.040,8400.820,reyyan hanımın ayağına kahvaltı gitsin,8l88wzheGsc
3,818.862,822.271,bin dokuz yüz seksen üç yılındaki seçimlerde t...,P0fKKr95Uu0
4,4130.274,4133.287,elleri kırılsın o cemilin oy nenem niye dövmüş,zXEY8j9Z0oI
...,...,...,...,...
3856314,27.893,28.711,benimle gel,dRl7KrgM9JI
3856315,237.933,238.994,halam o benim,fO8TyCMoDP8
3856316,891.940,898.300,giderken de veya bir yere yemeğe gitmişsin bir...,uLM3hC_A9B0
3856317,8705.880,8707.340,göreceğiz,8d2qntjg2GQ


In [13]:
df_word_group_select = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name= f"{sheets}")
df_word_group_select

,threegram
0,ama şimdi olmaz
1,asla olmaz demek
2,ben gerçekten üzgünüm
3,benimle geliyor musun
4,bu küçük kız
5,bunun için teşekkürler
6,bütün gece mi
7,evet sanırım onları
8,geri ver şunu
9,hadi al şunu


In [ ]:
# Option 
# Skip if it doesn't need
remove_videoid_list = list(set(df_word_group_select["video_id"]))
#remove_videoid_list

In [ ]:
len(remove_videoid_list)

In [ ]:
# Option
df_youtube_select = df_youtube_select[~df_youtube_select["video_id"].isin(remove_videoid_list)]
df_youtube_select

In [14]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[first_column:end_column]:
        string = df_word_group_select.loc[i,j]
        
        if pd.isnull(string) == False and string != 'nan':
            search_list.append(string.strip())
            search_list = list(set(search_list))  # for unique value
        else:
            pass

In [15]:
len(search_list)

24

In [16]:
df_search_result = word_group_youtube(df_youtube_select, search_list, "sentence", sample_num)
df_search_result

,search_string,start_time,end_time,sentence,video_id
0,yoksa sizi de,6189.135,6191.665,yoksa sizi de davet etmek isterdik ama,H5tCfLoE_UM
1,yoksa sizi de,6756.040,6760.640,haydi türkan hanım haydi yoksa sizi de öldürme...,f332DVN3EoQ
2,benimle geliyor musun,2185.052,2186.671,elif sen benimle geliyor musun,1OLV2bjYgA0
3,benimle geliyor musun,6075.580,6077.120,benimle geliyor musun candan,aWAp9pXi_DI
4,benimle geliyor musun,6543.215,6544.881,benimle geliyor musun gelmiyor musun,liz1ODAVzyk
5,benimle geliyor musun,16.902,18.722,benimle geliyor musun gelmiyor musun,sJ9F9o0JySU
6,benimle geliyor musun,6837.595,6839.085,benimle geliyor musun,awA8QjEncEU
7,bütün gece mi,8023.944,8025.652,bütün gece mi evet,P_KbNZi5uUo
8,bütün gece mi,3811.852,3813.102,bütün gece mi,UfpByy3jOCY
9,ol tamam mı,1548.140,1549.140,güçlü ol tamam mı,7VNGhuCG6ps


In [17]:
df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,yoksa sizi de,6189.135000,6190.067105,yoksa sizi de davet etmek isterdik ama,H5tCfLoE_UM
1,yoksa sizi de,6757.738462,6758.800000,haydi türkan hanım haydi yoksa sizi de öldürme...,f332DVN3EoQ
2,benimle geliyor musun,2185.483733,2186.671000,elif sen benimle geliyor musun,1OLV2bjYgA0
3,benimle geliyor musun,6075.580000,6076.790000,benimle geliyor musun candan,aWAp9pXi_DI
4,benimle geliyor musun,6543.215000,6544.233111,benimle geliyor musun gelmiyor musun,liz1ODAVzyk
5,benimle geliyor musun,16.902000,18.014222,benimle geliyor musun gelmiyor musun,sJ9F9o0JySU
6,benimle geliyor musun,6837.595000,6839.085000,benimle geliyor musun,awA8QjEncEU
7,bütün gece mi,8023.944000,8025.272444,bütün gece mi evet,P_KbNZi5uUo
8,bütün gece mi,3811.852000,3813.102000,bütün gece mi,UfpByy3jOCY
9,ol tamam mı,1548.434118,1549.140000,güçlü ol tamam mı,7VNGhuCG6ps


In [18]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,yoksa sizi de,6188.535000,6190.667105,yoksa sizi de davet etmek isterdik ama,H5tCfLoE_UM
1,yoksa sizi de,6757.138462,6759.400000,haydi türkan hanım haydi yoksa sizi de öldürme...,f332DVN3EoQ
2,benimle geliyor musun,2184.883733,2187.271000,elif sen benimle geliyor musun,1OLV2bjYgA0
3,benimle geliyor musun,6074.980000,6077.390000,benimle geliyor musun candan,aWAp9pXi_DI
4,benimle geliyor musun,6542.615000,6544.833111,benimle geliyor musun gelmiyor musun,liz1ODAVzyk
5,benimle geliyor musun,16.302000,18.614222,benimle geliyor musun gelmiyor musun,sJ9F9o0JySU
6,benimle geliyor musun,6836.995000,6839.685000,benimle geliyor musun,awA8QjEncEU
7,bütün gece mi,8023.344000,8025.872444,bütün gece mi evet,P_KbNZi5uUo
8,bütün gece mi,3811.252000,3813.702000,bütün gece mi,UfpByy3jOCY
9,ol tamam mı,1547.834118,1549.740000,güçlü ol tamam mı,7VNGhuCG6ps


In [19]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 

,search_string,start_time,end_time,sentence,video_id
0,yoksa sizi de,6189,6191,yoksa sizi de davet etmek isterdik ama,H5tCfLoE_UM
1,yoksa sizi de,6757,6759,haydi türkan hanım haydi yoksa sizi de öldürme...,f332DVN3EoQ
2,benimle geliyor musun,2185,2187,elif sen benimle geliyor musun,1OLV2bjYgA0
3,benimle geliyor musun,6075,6077,benimle geliyor musun candan,aWAp9pXi_DI
4,benimle geliyor musun,6543,6545,benimle geliyor musun gelmiyor musun,liz1ODAVzyk
5,benimle geliyor musun,16,19,benimle geliyor musun gelmiyor musun,sJ9F9o0JySU
6,benimle geliyor musun,6837,6840,benimle geliyor musun,awA8QjEncEU
7,bütün gece mi,8023,8026,bütün gece mi evet,P_KbNZi5uUo
8,bütün gece mi,3811,3814,bütün gece mi,UfpByy3jOCY
9,ol tamam mı,1548,1550,güçlü ol tamam mı,7VNGhuCG6ps


In [20]:
df_word_group_time_loc_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result['video_id'].map(str)+"&t="+df_word_group_time_loc_result['start_time'].map(str)+"s"
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,yoksa sizi de,6189,6191,yoksa sizi de davet etmek isterdik ama,H5tCfLoE_UM,https://www.youtube.com/watch?v=H5tCfLoE_UM&t=...
1,yoksa sizi de,6757,6759,haydi türkan hanım haydi yoksa sizi de öldürme...,f332DVN3EoQ,https://www.youtube.com/watch?v=f332DVN3EoQ&t=...
2,benimle geliyor musun,2185,2187,elif sen benimle geliyor musun,1OLV2bjYgA0,https://www.youtube.com/watch?v=1OLV2bjYgA0&t=...
3,benimle geliyor musun,6075,6077,benimle geliyor musun candan,aWAp9pXi_DI,https://www.youtube.com/watch?v=aWAp9pXi_DI&t=...
4,benimle geliyor musun,6543,6545,benimle geliyor musun gelmiyor musun,liz1ODAVzyk,https://www.youtube.com/watch?v=liz1ODAVzyk&t=...
5,benimle geliyor musun,16,19,benimle geliyor musun gelmiyor musun,sJ9F9o0JySU,https://www.youtube.com/watch?v=sJ9F9o0JySU&t=16s
6,benimle geliyor musun,6837,6840,benimle geliyor musun,awA8QjEncEU,https://www.youtube.com/watch?v=awA8QjEncEU&t=...
7,bütün gece mi,8023,8026,bütün gece mi evet,P_KbNZi5uUo,https://www.youtube.com/watch?v=P_KbNZi5uUo&t=...
8,bütün gece mi,3811,3814,bütün gece mi,UfpByy3jOCY,https://www.youtube.com/watch?v=UfpByy3jOCY&t=...
9,ol tamam mı,1548,1550,güçlü ol tamam mı,7VNGhuCG6ps,https://www.youtube.com/watch?v=7VNGhuCG6ps&t=...


In [21]:
search_result_num = df_word_group_time_loc_result["search_string"].nunique()
search_result_num

10

In [22]:
search_result_set = set(df_word_group_time_loc_result["search_string"])
search_all_set = set(search_list)

In [23]:
search_all_set.difference(search_result_set)

{'ben gerçekten üzgünüm',
 'bunun için teşekkürler',
 'evet sanırım onları',
 'geri ver şunu',
 'hadi ama dostum',
 'haydi git buradan',
 'herkes kendi geliyor',
 'hey neler oluyor',
 'oh hadi ama',
 'onunla birlikte değilim',
 'pekala hadi bakalım',
 'selam baba merhaba',
 'tabii eğer o',
 'şimdi tekrar söyle'}

In [24]:
df_result_word_manuel_count = word_count_result(df_word_group_time_loc_result, df_word_group_time_loc_result.columns[0:1], set_condition=True)
df_result_word_manuel_count

,word,word_count
0,mi,2
2,olmaz,2
3,gece,2
1,yoksa,2
15,küçük,1
24,asla,1
23,benimle,1
22,bizi,1
21,bu,1
20,bütün,1


In [ ]:
# # Option merge
# df_word_count_merge = pd.merge(df_result_word_manuel_count, df_word_select[["word"]], how="inner", on="word")
# df_word_count_merge.drop_duplicates(inplace=True)
# df_word_count_merge.sort_values(by="word_count", ascending=False, inplace=True)
# df_word_count_merge.reset_index(drop=True, inplace=True)
# df_word_count_merge

In [25]:
set_word = set(df_word_select["word"])
#set_count = set(df_word_count_merge["word"])  # Option merge
set_count = set(df_result_word_manuel_count["word"])

In [ ]:
set_word.difference(set_count)

In [26]:
df_word_group_time_loc_result.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [ ]:
# df_result_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
# {sample_num}_Youtube_{time_shift}s_Timeshift_Result_Word_Count.xlsx", index=False) 

#### Copy Move And Delete

In [27]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_Result*.xlsx")
output_file

['Turkish_67_Threegram_Remain_Word_10_Youtube_0.6s_Timeshift_Result2.xlsx',
 'Turkish_10_Threegram_Remain_Word2_10_Youtube_0.6s_Timeshift_Result.xlsx']

In [ ]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass

#### Temp

In [ ]:
set_result = set(df_word_group_time_loc_result["search_string"].unique())
set_selected = set(search_list)

In [ ]:
df_not_in_result = pd.DataFrame(set_selected.difference(set_result), columns=["not_in_youtube_result"])
df_not_in_result

In [ ]:
not_in_result_num = len(df_not_in_result)
not_in_result_num

In [ ]:
#df_not_in_result.to_excel(f"{lang_folder.capitalize()}_{not_in_result_num}_{file_ext}_\
#Not_In_{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [ ]:
search_list2 = df_not_in_result["not_in_youtube_result"].to_list()

In [ ]:
df_search_result2 = word_group_youtube(df_youtube_sentence, search_list2, "sentence", sample_num)
df_search_result2

In [ ]:
search_result2_num = df_search_result2["search_string"].nunique()
search_result2_num

In [ ]:
df_word_group_time_loc_result2 = word_group_time_loc(df_search_result2, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result2

In [ ]:
df_word_group_time_loc_result2.start_time = df_word_group_time_loc_result2.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result2.end_time = df_word_group_time_loc_result2.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result2

In [ ]:
df_word_group_time_loc_result2["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result2['video_id'].map(str)+"&t="+df_word_group_time_loc_result2['start_time'].map(str)+"s"
df_word_group_time_loc_result2

In [ ]:
search_result_num2 = df_word_group_time_loc_result2["search_string"].nunique()
search_result_num2

In [ ]:
df_word_group_time_loc_result2.to_excel(f"{lang_folder.capitalize()}_{search_result_num2}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result2.xlsx", index=False) 